In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-09-28 10:12:39.113656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 10:12:39.637323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [22]:
model = SAT1LSTM(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [21]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [23]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

Epoch 1/20
765/765 [==============================] - 26s 27ms/step - loss: 0.8509 - accuracy: 0.6729 - val_loss: 0.6442 - val_accuracy: 0.7540
Epoch 2/20
765/765 [==============================] - 20s 25ms/step - loss: 0.6619 - accuracy: 0.7502 - val_loss: 0.6111 - val_accuracy: 0.7687
Epoch 3/20
765/765 [==============================] - 20s 25ms/step - loss: 0.6239 - accuracy: 0.7643 - val_loss: 0.5586 - val_accuracy: 0.7866
Epoch 4/20
765/765 [==============================] - 20s 25ms/step - loss: 0.6036 - accuracy: 0.7777 - val_loss: 0.5653 - val_accuracy: 0.7816
Epoch 5/20
765/765 [==============================] - 20s 25ms/step - loss: 0.5856 - accuracy: 0.7768 - val_loss: 0.5794 - val_accuracy: 0.7744
Epoch 6/20
765/765 [==============================] - 20s 25ms/step - loss: 0.5772 - accuracy: 0.7829 - val_loss: 0.6057 - val_accuracy: 0.7913
Epoch 7/20
765/765 [==============================] - 20s 25ms/step - loss: 0.5634 - accuracy: 0.7906 - val_loss: 0.5481 - val_accuracy:

(<keras.src.callbacks.History at 0x7f7512c4a9d0>, None)